In [1]:
import os
import collections

from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Conv1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nltk
import numpy as np
import codecs

np.random.seed(42)

INPUT_FILE = os.path.join(os.path.dirname("__file__"),
                          "data/training.txt")
LOG_DIR = os.path.join(os.path.dirname("__file__"), "logs")
VOCAB_SIZE = 5000
EMBED_SIZE = 100
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 64
NUM_EPOCHS = 20

Using TensorFlow backend.


In [2]:
counter = collections.Counter()
# file open
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as fin:
    maxlen = 0
    for line in fin:
        # 行をtabでsplit
        _, sent = line.strip().split("\t")
        try:
            # NLTK(Natural Language Toolkit)で単語分割
            words = [x.lower() for x in nltk.word_tokenize(sent)]
        except LookupError:
            print("English tokenize does not downloaded. So download it.")
            nltk.download("punkt")
            words = [x.lower() for x in nltk.word_tokenize(sent)]
        maxlen = max(maxlen, len(words))
        for word in words:
            # 単語ごとでカウント
            counter[word] += 1
            # int型でdictionaryを初期化
            word2index = collections.defaultdict(int)
            # 出現回数が多い順に（文字, 回数）のタプル5000個に対して単語IDを定義
            for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
                word2index[word[0]] = wid + 1
            vocab_sz = len(word2index) + 1
            index2word = {v: k for k, v in word2index.items()}

In [3]:
xs, ys = [], []
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as fin:
    for line in fin:
        label, sent = line.strip().split("\t")
        ys.append(int(label))
        words = [x.lower() for x in nltk.word_tokenize(sent)]
        wids = [word2index[word] for word in words]
        xs.append(wids)

X = pad_sequences(xs, maxlen = maxlen)
Y = np_utils.to_categorical(ys)

In [5]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size = 0.3, random_state = 42)

# ネットワークの作成
model = Sequential()
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length = maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(filters = NUM_FILTERS, kernel_size = NUM_WORDS, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation = "softmax"))

# コンパイル
model.compile(optimizer = "adam", loss="categorical_crossentropy", metrics = ["accuracy"])
history = model.fit(Xtrain, Ytrain, batch_size = BATCH_SIZE,
                    epochs = NUM_EPOCHS,
                    # エラーになるのでコメントアウト
                    # callbacks = [TensorBoard(LOG_DIR)],
                    validation_data = (Xtest, Ytest))

C:\Users\tetsu\Anaconda3\envs\py_gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4960 samples, validate on 2126 samples
Epoch 1/20
4960/4960 [==============================] - 2s 328us/step - loss: 0.2984 - accuracy: 0.8782 - val_loss: 0.0401 - val_accuracy: 0.9845
Epoch 2/20
4960/4960 [==============================] - 1s 274us/step - loss: 0.0223 - accuracy: 0.9935 - val_loss: 0.0226 - val_accuracy: 0.9925
Epoch 3/20
4960/4960 [==============================] - 1s 272us/step - loss: 0.0075 - accuracy: 0.9982 - val_loss: 0.0180 - val_accuracy: 0.9929
Epoch 4/20
4960/4960 [==============================] - 1s 268us/step - loss: 0.0034 - accuracy: 0.9996 - val_loss: 0.0168 - val_accuracy: 0.9953
Epoch 5/20
4960/4960 [==============================] - 1s 266us/step - loss: 0.0016 - accuracy: 0.9998 - val_loss: 0.0176 - val_accuracy: 0.9958
Epoch 6/20
4960/4960 [==============================] - 1s 268us/step - loss: 0.0016 - accuracy: 0.9998 - val_loss: 0.0171 - val_accuracy: 0.9944
Epoch 7/20
4960/4960 [==============================] - 1s 268us/step - loss